# Preparing the environment

## Import dependencies

In [1]:
#    Spark
from pyspark import SparkContext
#    Spark Streaming
from pyspark.streaming import StreamingContext
#    Kafka
from pyspark.streaming.kafka import KafkaUtils
#    json parsing
import json
#    print function
from __future__ import print_function
#    os
import sys

## Create Streaming Context

In [2]:
zkQuorum = "localhost:2181"
topic = "twitter-stream"
seconds_to_run = 60
ssc = StreamingContext(sc, seconds_to_run)

tweets = KafkaUtils.createStream(ssc, zkQuorum, "spark-streaming-consumer", {topic: 1})

## Parse the inbound message as json

The inbound stream is a DStream, which supports various built-in transformations such as map which is used here to parse the inbound messages from their native JSON format.
Note that this will fail horribly if the inbound message isn't valid JSON.

In [3]:
parsed = tweets.map(lambda v: json.loads(v[1]))

In [4]:
parsed.count().map(lambda x:'Tweets in this batch: %s' % x).pprint()

## Extract Text from each tweet

In [5]:
text_dstream = parsed.map(lambda tweet: tweet['text'])
#text_dstream.pprint()

## Process the extracted text 

In [6]:
import nltk
import csv
import string
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [7]:
def filter_out_unicode(tweet):
  
    try:
        clean_tweet = str(tweet)
    except UnicodeEncodeError:
        pass
    return clean_tweet

def expand_around_chars(text, characters):
    for char in characters:
        text = text.replace(char, ' ' + char + ' ')
    return text

def strip_quotations_newline(text):
    clean_tweet = ' '.join(text.split())
    clean_tweet = clean_tweet.encode('utf-8')
    clean_tweet = clean_tweet.replace('",\'','')
    clean_tweet = re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', "", clean_tweet)
    clean_tweet = re.sub(r'''(@[A-Za-z0-9]+)''', "", clean_tweet)
    clean_tweet = re.sub("([0-9]+)", "", clean_tweet)
    clean_tweet = re.sub(r'[^\x00-\x7F]+','', clean_tweet)
    return clean_tweet

def split_text(text):
    text = strip_quotations_newline(text)
    text = expand_around_chars(text, '\/".,()&[]{}:;!-_\'')
    splitted_text = text.split(' ')
    cleaned_text = [x for x in splitted_text if len(x) > 2]
    text_lowercase = [x.lower() for x in cleaned_text]
    return text_lowercase

In [8]:
mess = text_dstream.map(lambda text: split_text(text))
mess.pprint()

## Create clean tweet

## Convert data to be classified

In [17]:
from pyspark.ml.feature import HashingTF, IDF
hashingTF = HashingTF(numFeatures=50000)
fh

In [19]:
def classify_tweet(tf):
    idf = IDF().fit(tf)
    return idf

## Start the streaming context

In [11]:
ssc.start()
ssc.awaitTermination(timeout=180)

-------------------------------------------
Time: 2017-06-20 14:49:00
-------------------------------------------
Tweets in this batch: 97

-------------------------------------------
Time: 2017-06-20 14:49:00
-------------------------------------------
stay the fuck outta their hood then
any teenation member beat quake champions will recieve till reach you lose will
panther whales out here hittin licks seen all
ara won shaking #whatsmynamestwin
global unrest event underway west coast usa canada hit earthquake
jonny amp thom play one off show macerata sferisterio august proceeds will help the restoration earthq
after you predicted crooked hilly landslide you now are saying maybe and maybe not? lol
blackpink shaking #krunk
scoffs shaking her head please outside often both know that
jay panorama reported people lived that tower survivors are really suggesting only are dead
...



Py4JJavaError: An error occurred while calling o33.awaitTerminationOrTimeout.
: org.apache.spark.SparkException: An exception was raised by Python:
Traceback (most recent call last):
  File "/usr/local/src/spark/spark-2.0.2-bin-hadoop2.7/python/pyspark/streaming/util.py", line 65, in call
    r = self.func(t, *rdds)
  File "/usr/local/src/spark/spark-2.0.2-bin-hadoop2.7/python/pyspark/streaming/dstream.py", line 159, in <lambda>
    func = lambda t, rdd: old_func(rdd)
  File "<ipython-input-10-977c4d20cde0>", line 1, in <lambda>
    tweet_df = mess.foreachRDD(lambda rdd: rdd.toDF("text"))
  File "/usr/local/src/spark/spark-2.0.2-bin-hadoop2.7/python/pyspark/sql/session.py", line 57, in toDF
    return sparkSession.createDataFrame(self, schema, sampleRatio)
  File "/usr/local/src/spark/spark-2.0.2-bin-hadoop2.7/python/pyspark/sql/session.py", line 492, in createDataFrame
    schema = _parse_datatype_string(schema)
  File "/usr/local/src/spark/spark-2.0.2-bin-hadoop2.7/python/pyspark/sql

-------------------------------------------
Time: 2017-06-20 14:50:00
-------------------------------------------
Tweets in this batch: 61

-------------------------------------------
Time: 2017-06-20 14:50:00
-------------------------------------------
the whales are fed with our shit
she shaking them titties girl yes
gangs aggressive killer whales are shaking down alaska fishing boats for their fish report
imagine this dance king hit the stage already shaking
aggressive shaking down
your area expertise sir
got shaking hands during #mmvas slow hands
ara won shaking #whatsmynamestwin
your polls have never been close being right they were hillary would won landslide fire your polling agent lol
residents flee after tsunami hit greenland following earthquake
...

-------------------------------------------
Time: 2017-06-20 14:51:00
-------------------------------------------
Tweets in this batch: 41

-------------------------------------------
Time: 2017-06-20 14:51:00
-------------------

## Sort the author count

In [8]:
author_counts_sorted_dstream = author_counts.transform(\
  (lambda foo:foo\
   .sortBy(lambda x:( -x[1]))))

In [9]:
author_counts_sorted_dstream.pprint()

## Get top 5 authors by tweet count

In [10]:
top_five_authors = author_counts_sorted_dstream.transform\
  (lambda rdd:sc.parallelize(rdd.take(5)))
top_five_authors.pprint()

## Get authors with more than one tweet, or whose username starts with 'a'

In [13]:
filtered_authors = author_counts.filter(lambda x:\
                                                x[1]>1 \
                                                or \
                                                x[0].lower().startswith('rm'))

In [14]:
filtered_authors.transform\
  (lambda rdd:rdd\
  .sortBy(lambda x:-x[1]))\
  .pprint()

## List the most common words in the tweets

In [15]:
parsed.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x:-x[1]))\
    .pprint()